In [1]:
import os
import re
from collections import Counter
os.system("pip install gensim matplotlib scikit-learn pandas")

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.4/159.4 kB 3.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 46.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 60.5 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 60.8 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 51.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.7/310.7 kB 4.6 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 39.3 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 3.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.2


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


0

In [2]:
import gensim
import gensim.downloader
import gensim.downloader as api

import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import PCA

In [3]:
model_name = "english_word2vec.bin"
model_path = os.path.join("..","in","model",model_name)
model = gensim.models.KeyedVectors.load(model_path)

In [4]:
#data
data_name = "data_spoty.csv"
data_path = os.path.join("..","in","data",data_name)
data = pd.read_csv(data_path)

In [5]:
#subset
singer = "ABBA"
sub_data = data.loc[data['artist'] == singer]

In [6]:
#embed
query_word = "love"
n_similar = 12
query_array = model.most_similar(query_word,topn=n_similar)
query_array.append((query_word, 1))

In [9]:
# find in subset
for word_value_pair in query_array:
    target_word = word_value_pair[0]
    occurence_count = {}
    for row in sub_data.loc:
        song_title = row["song"]
        song_lyric = row["text"]
        #clean lyrics to be compatible with the word embedding labels
        clean_lyric = re.sub(r"""
               [,.;@#?!&$\n]+  # Accept one or more copies of punctuation
               \ *           # plus zero or more copies of a space,
               """,
               " ",          # and replace it with a single space
               song_lyric.lower(), flags=re.VERBOSE)
        
        #count list elements in lyric
        n_occurences = clean_lyric.split().count(target_word)
        temporary_dict = {target_word:n_occurences}
        occurence_count.update(temporary_dict)
                


    


KeyError: 113

In [69]:
# find in subset (subset rows by indexing row number to sub_data.loc)
max_row = len(sub_data.index)
occurence_count_full = {"song":[]}
for row_index in range(0,max_row):
    row = sub_data.loc[row_index]
    song_title = row["song"]
    song_lyric = row["text"]
    artist = row["artist"]
    #clean lyrics to be compatible with the word embedding labels
    clean_lyric = re.sub(r"""
           [,.;@#?!&$\n]+  # Accept one or more copies of punctuation
           \ *           # plus zero or more copies of a space,
           """,
           " ",          # and replace it with a single space
           song_lyric.lower(), flags=re.VERBOSE)
    
    #for each word in list
    occurence_count = {}
    for word_value_pair in query_array:
        target_word = word_value_pair[0]
        #count list elements in lyric
        n_occurences = clean_lyric.split().count(target_word)
        temporary_dict = {artist:song_title,target_word:n_occurences}
        #check counts in the dictionary. if none found, it isn't saved
        non_zero_counter = 0
        for count in temporary_dict.values():
            if type(count) == int: #the first is the title so that is a str

                if count > 0:
                    non_zero_counter += 1
                else:
                    pass

            else:
                pass

        if non_zero_counter > 0:
            occurence_count.update(temporary_dict)
        else:
            pass

    #append dict if not empty
    if len(occurence_count) > 0:     
        occurence_count_full["song"].append(occurence_count)
    else:
        pass


    


In [68]:
temporary_dict

{'ABBA': 'You Owe Me One', 'love': 0}

In [70]:
occurence_count_full

{'song': [{'ABBA': 'As Good As New', 'love': 7},
  {'ABBA': 'Bang', 'love': 10},
  {'ABBA': 'Bang-A-Boomerang', 'love': 10},
  {'ABBA': 'Chiquitita', 'hate': 1},
  {'ABBA': 'Dance', 'love': 2},
  {'ABBA': 'Disillusion', 'love': 1},
  {'ABBA': 'Dream World', 'love': 2},
  {'ABBA': 'Eagle', 'love': 1},
  {'ABBA': 'From A Twinkling Star To A Passing Angel', 'love': 2},
  {'ABBA': 'Gimme Gimme Gimme', 'hate': 1},
  {'ABBA': 'Gonna Sing You My Lovesong', 'loving': 2, 'love': 10},
  {'ABBA': 'Hamlet III', 'loved': 1},
  {'ABBA': 'Happy Hawaii', 'love': 2},
  {'ABBA': 'He Is Your Brother', 'love': 6},
  {'ABBA': 'Head Over Heels', 'love': 1},
  {'ABBA': "Here We'll Stay", 'love': 11},
  {'ABBA': 'Honey, Honey', 'love': 2},
  {'ABBA': 'I Am Just A Girl', 'loves': 1, 'love': 4},
  {'ABBA': 'I Do, I Do, I Do, I Do, I Do', 'love': 6},
  {'ABBA': 'I Saw It In The Mirror', 'love': 1},
  {'ABBA': "I've Been Waiting For You", 'adore': 2, 'love': 3},
  {'ABBA': 'Kisses Of Fire', 'love': 3},
  {'ABBA':

In [54]:
occurence_count.values()

dict_values(['You Owe Me One', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
# van-e, melyik, hányszor
# undying_love???
#cs (Artist,title,all_freq/all_songs,ind_count,ind_freq/all_songs)